In [8]:
import yaml
import os
config = yaml.load(open('configs/config.yaml'))
def full_path(file):

    '''
    returns the full path to a reference
    '''

    build = config['ref']['build']
    full_ref_path = os.path.join(config['paths']['mystatic'], config['ref'][build][file])
    return full_ref_path

### get_bed_file

In [26]:
def get_bed_file(config, covered_or_padded):
    '''
    returns the appropriate bed_file depending on build and library
    '''
    
    build = config['ref']['build']
    lib = config['setup']['library']
    lib_version = config['setup']['library_version']
    prefix = 'SS' if lib == 'SureSelect' else lib
    suffix = '_nochr' if build == 'hg19' else ''
    bed_name = f"{prefix}_{lib_version}_{build}_{covered_or_padded}{suffix}.bed"
    bed_file = os.path.join(config['paths']['mystatic'], 'bed_files', lib, build,bed_name)
    return bed_file

get_bed_file(config, 'Padded')

'/fast/groups/ag_damm/work/ref/bed_files/SureSelect/hg38/SS_HAEv7_hg38_Padded.bed'

### filter annovar protocol

In [61]:
# reduce list
build_files = []
for file in file_list:
    if build in file:
        build_files.append(file)
        
anno_list = []
missing = []
for anno in anno_refs:
    for file in build_files:
        if anno in file:
            anno_list.append(anno)
            break
    else:
        missing.append(anno)
protocol = ','.join(anno_list)

operation_list = []
for anno in anno_list:
    if anno == "refGene":
        operation_list.append('g')
    elif anno in ['cytoBand', 'genomicSuperDups']:
        operation_list.append('r')
    else:
        operation_list.append('f')
operation = ','.join(operation_list)

missing
protocol
operation

# build_files

['genomicSuperDups',
 'esp6500_all',
 '1000g2010nov_all',
 '1000g2014oct_all',
 'snp131',
 'snp131NonFlagged',
 'snp138',
 'snp138NonFlagged',
 'avsnp138',
 'ljb23_pp2hdiv',
 'ljb23_sift',
 'spidex',
 'icgc21']

'refGene,cytoBand,esp6500siv2_all,dbscsnv11,avsnp150,cosmic70,clinvar_20150629,clinvar_20170905,clinvar_20190305,ljb26_all'

'g,r,f,f,f,f,f,f,f,f'

### filter

In [68]:
filter_name = 'daniel'
filter_config = config['filter'][filter_name]
def get_filter(f):
    '''
    takes the filter dictionary and returns full path to the filter script using filter['name']
    '''

    return os.path.join(config['snakedir'], config['paths']['scripts'], 'filters', f['path'])
filter_config

In [70]:
filter_script = get_filter(filter_config)
filter_script

'/fast/users/szyskam_c/snakes/projects/AML/scripts/filters/daniel.py'

In [78]:
[[f"-{param}", f"{val}"] for param, val in filter_config['params'].items()]

[['-TM2_limit', '4']]

In [71]:
params_list = [param for plist in [[f"-{param}", f"{val}"] for param, val in filter_config['params'].items()] for param in plist]
params_list

['-TM2_limit', '4']

In [73]:
filter_cmd = [filter_script, *params_list, 'input_file', 'output']

In [75]:
print(f" ".join(filter_cmd))

/fast/users/szyskam_c/snakes/projects/AML/scripts/filters/daniel.py -TM2_limit 4 input_file output
